In [1]:
from google.cloud import vision
import os
import json
import re

In [2]:



def setup_google_credentials(json_path):
    """
    Set up the environment variable for Google Cloud authentication.

    Args:
        json_path (str): Path to the service account JSON key file.
    """
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = json_path


def ocr_image(image_path):
    """
    Perform OCR on an image using Google Cloud Vision API.

    Args:
        image_path (str): Path to the image file.

    Returns:
        str: Extracted text from the image.
    """
    # Initialize the Vision client
    client = vision.ImageAnnotatorClient()

    # Load the image
    with open(image_path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)

    # Perform text detection
    response = client.text_detection(image=image)

    # Check for errors
    if response.error.message:
        raise Exception(f"Vision API Error: {response.error.message}")

    # Extract text
    text = response.text_annotations[0].description if response.text_annotations else "No text found."
    return text

In [3]:
setup_google_credentials("glowing-cooler-441113-m8-98ea36811795.json")

In [6]:
import os
import json

def GetVietnameseText(input_folder_path, output_folder_path, filename, startindex, endindex, output_name):
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)
    
    data_result = {}
    for i in range(startindex, endindex + 1):
        filepath = os.path.join(input_folder_path, f"{filename}{i}.png")
        if os.path.exists(filepath):
            text_result = ocr_image(filepath)
            
            data_result[f"{filename}{i}"] = text_result
        else:
            print(f"File không tồn tại: {filepath}")
    
    annotations_path = os.path.join(output_folder_path, output_name)
    if os.path.exists(annotations_path):
        with open(annotations_path, "r", encoding="utf-8") as f:
            existing_data = json.load(f)
    else:
        existing_data = {}
    
    existing_data.update(data_result)

    with open(annotations_path, "w", encoding="utf-8") as f:
        json.dump(existing_data, f, ensure_ascii=False, indent=4)

    print("Dữ liệu đã được lưu thành công!")


# **OCR image**

In [7]:
GetVietnameseText(r"D:\NLP_Project\data_prepare\NLP_Project_Group6\output\Khanh_QN_pages_2",
                  r"D:\NLP_Project\data_prepare\NLP_Project_Group6\output\vietnamese_label",
                  r"Khanh_TNYL_P2_QN_page_", 1, 22, "annotations_P2.json")

Dữ liệu đã được lưu thành công!


In [10]:
GetVietnameseText(r"D:\NLP_Project\data_prepare\NLP_Project_Group6\output\Khanh_QN_pages",
                  r"D:\NLP_Project\data_prepare\NLP_Project_Group6\output\vietnamese_label",
                  r"Khanh_TNYL_P1_QN_page_", 1, 25, "annotations_2.json")

Dữ liệu đã được lưu thành công!


# **Clean Data**

In [8]:
import json
import re


file_path = ".\\output\\vietnamese_label\\annotations.json"
output_path = ".\\output\\vietnamese_label\\annotations_cleaned.json"

file2_path = ".\\output\\vietnamese_label\\annotations_3.json"
output2_path = ".\\output\\vietnamese_label\\annotations_3_cleaned.json"

words_to_remove = [
    "COLLEGE LIDIARY",
    "MANU LLEGE!!!!UARY",
    "MEUL LIIVINE",
    "COLLLULLIDIVAN",
    "COLLEGE LIDKARY",
    "ten Oula Ngi",
    "Ouba Nau",
    "Outa Na",
    "Aa Nair",
    "The Chung n",
    "Chond Out Non",
    "W Chung Quốa Não",
    "Tứ Chung Quốc Ngữ",
    "Tứ Chung Quốc Nơi",
    "Tứ Chung Cu Nơi",
    "Tứ Chung Quốc Ngũ",
    "Tử Chung Quốc Ngữ",
    "Tú Chung Quốc Ngữ",
    "ý Chung Quốc Ngữ",
    "Chung Quốc Ngữ",
    "Tử Nguyên Yếu Lý Phần I",
    "Tứ Nguyên Yếu Lý Phần",
    "ཏེ .",
    "י I I",
    "LIARIÐ G. PQ WBG ",
    "HARVAR % DR RAGI & BERARY ",
    "I I : ",
    "PARVA&R_ALGI B BRARY",
    "י I PARVA . < B BRARY : "
]


def clean_data(filepath, outputpath):
    with open(filepath, "r", encoding="utf-8") as file:
        data = json.load(file)


    cleaned_data = {}
    for key, value in data.items():
        # Xóa các từ cụ thể
        
        # Xóa toàn bộ dấu ngoặc tròn
        value = re.sub(r"[()]", "", value)

        value = re.sub(r"(?<=[\wÀ-ỹ])-+(?=[\wÀ-ỹ])", " ", value)
        # Xóa dấu gạch ngang
        value = re.sub(r"-", "", value)
        # Xóa các số
        value = re.sub(r"\d+", "", value)
        # Xóa khoảng trắng thừa
        value = re.sub(r"\s+", " ", value).strip()
        
        for word in words_to_remove:
            value = value.replace(word, "")

        # Lưu kết quả vào cleaned_data
        cleaned_data[key] = value

    # Ghi dữ liệu đã làm sạch vào file mới
    with open(outputpath, "w", encoding="utf-8") as file:
        json.dump(cleaned_data, file, ensure_ascii=False, indent=4)

    print(f"Dữ liệu đã làm sạch được lưu tại: {outputpath}")


In [9]:
clean_data(file2_path, output2_path)

Dữ liệu đã làm sạch được lưu tại: .\output\vietnamese_label\annotations_3_cleaned.json
